In [2]:
%matplotlib inline
import pandas as pd
import numpy as np

adult = pd.read_csv("adult_with_pii.csv")
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)
def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

本地差分隐私

截至目前，我们只考虑了差分隐私的中心模型（Central Model）。在中心模型中，原始敏感数据被汇总到单个数据集中。在这种场景下，我们假定分析者是恶意的，但存在一个可信任的数据管理者，由它持有数据集并能正确执行分析者指定的差分隐私机制。

这种设定通常是不现实的。在很多情况下，数据管理者和分析者是同一个人，且实际上不存在一个可信第三方，能由它持有数据集并执行差分隐私机制。事实上，往往是我们不信任的组织来收集我们最敏感的数据。这样的组织显然无法成为可信数据管理者。

中心差分隐私模型的一种替代方案是差分隐私本地模型（Local Model）。在本地模型中，数据在离开数据主体控制之前就已经满足差分隐私。例如，在将数据发送给数据管理者之前，用户就在自己的设备上为自己的数据添加噪声。在本地模型中，数据管理者不需要是可信的，因为他们收集的是已经满足差分隐私的数据。

因此，相比于中心模型，本地模型有着巨大的优势：数据主体不需要相信除他们自己以外的任何人。这一优势使得本地模型在实际系统中有着广泛的应用，包括谷歌和苹果都部署了基于本地模型的差分隐私应用。

不幸的是，本地模型也有明显的缺点：在相同的隐私消耗量下，对于相同的问询，本地模型问询结果的准确性通常比中心模型低几个数量级。这种巨大的准确性损失意味着只有较少类型的问询适用于本地差分隐私。即便如此，只有当数据量较大（即参与者数量较多时）时，差分隐私本地模型分析结果的准确率才可以满足实际要求。

本章，我们将学习两种本地差分隐私机制。第一种是随机应答（Randomized Response），第二种是一元编码（Unary Encoding）。

随机应答

In [3]:
def rand_resp_sales(response):
    truthful_response = response == 'Sales'
    
    # 第一次抛掷硬币
    if np.random.randint(0, 2) == 0:
        # 如实回答
        return truthful_response
    else:
        # （用第二次硬币抛掷结果）随机应答
        return np.random.randint(0, 2) == 0

让我们来询问200名从事销售工作的人，请他们使用随机应答算法回答此问题，看看结果如何。

In [4]:
pd.Series([rand_resp_sales('Sales') for i in range(200)]).value_counts()

True     153
False     47
Name: count, dtype: int64

In [5]:
responses = [rand_resp_sales(r) for r in adult['Occupation']]

In [6]:
pd.Series(responses).value_counts()

False    22660
True      9901
Name: count, dtype: int64

这次，我们得到的"否"数量比"是"数量更多。稍加思考，就会发现这是一个合理的统计结果，因为数据集中大多数参与者的职位都不是销售。

现在的关键问题是：我们如何根据这些回复，估计出数据集中销售人员的真实人数呢？"是"的数量并不能很好地估计销售人员数量：

In [7]:
len(adult[adult['Occupation'] == 'Sales'])

3650

In [8]:
responses = [rand_resp_sales(r) for r in adult['Occupation']]

# 我们估计出有1/4的"是"回复完全来自于硬币的随机抛掷结果
# 这些都是假的"是"
fake_yeses = len(responses)/4

# 回复为"是"的总人数
num_yeses = np.sum([1 if r else 0 for r in responses])

# 真实"是"的人数等于回复为"是"的总人数减去假"是"的人数
true_yeses = num_yeses - fake_yeses

In [9]:
# 用true_yesses估计"真实"组中回答"是"的人数
# 我们把人数翻倍，估计出回复为"是"的总人数
rr_result = true_yeses*2
rr_result

3861.5

In [10]:
true_result = np.sum(adult['Occupation'] == 'Sales')
true_result

3650

In [11]:
pct_error(true_result, rr_result)

5.794520547945205

当总人数相对比较大时，（例如，本例的总人数超过了3000），我们通常可以使用此方法得到一个错误率"可接受"的统计结果。此例子中的错误率低于5%。如果我们的目标是统计最受欢迎的职位，这个方法可以帮助我们得到较为准确的结果。然而，统计结果的错误率会随着总人数的降低而快速增大。

此外，随机应答的准确率和中心模型拉普拉斯机制的准确率相比要差出几个数量级。让我们使用此例子比较一下这两种机制：

In [12]:
pct_error(true_result, laplace_mech(true_result, 1, 1))

0.02410391432662536

确实存在效果更好的本地模型算法。然而，本地模型存在天生的限制条件：必须在提交数据前增加噪声。这意味着本地模型算法的准确率总是比最好的中心模型算法准确率低。

一元编码

In [13]:
domain = adult['Occupation'].dropna().unique()
domain

array(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners',
       'Prof-specialty', 'Other-service', 'Sales', 'Craft-repair',
       'Transport-moving', 'Farming-fishing', 'Machine-op-inspct',
       'Tech-support', 'Protective-serv', 'Armed-Forces',
       'Priv-house-serv'], dtype=object)

该技术的名称来源于所用的编码方法：如果应答域大小为k，我们将每个应答值编码为长度为k的比特向量。除了应答者的职位所对应的比特值为1以外，所有其他位置的编码均为0。机器学习领域称这种表示方法"独热编码"（One-hot Encoding）。

举例来说，'销售'是应答域中的第6个元素，因此'销售'职位的编码是第6个比特为1、其余比特值均为0的向量。

In [14]:
def encode(response):
    return [1 if d == response else 0 for d in domain]

encode('Sales')

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
def perturb(encoded_response):
    return [perturb_bit(b) for b in encoded_response]

def perturb_bit(bit):
    p = .75
    q = .25

    sample = np.random.random()
    if bit == 1:
        if sample <= p:
            return 1
        else:
            return 0
    elif bit == 0:
        if sample <= q:
            return 1
        else: 
            return 0

perturb(encode('Sales'))

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1]

In [16]:
def unary_epsilon(p, q):
    return np.log((p*(1-q)) / ((1-p)*q))

unary_epsilon(.75, .25)

2.1972245773362196

最后一步是聚合。如果我们没有对应答值进行过任何扰动，我们可以简单地对所有得到的应答向量逐比特相加，得到应答域中每个元素的计数结果：

In [17]:
counts = np.sum([encode(r) for r in adult['Occupation']], axis=0)
list(zip(domain, counts))

[('Adm-clerical', 3770),
 ('Exec-managerial', 4066),
 ('Handlers-cleaners', 1370),
 ('Prof-specialty', 4140),
 ('Other-service', 3295),
 ('Sales', 3650),
 ('Craft-repair', 4099),
 ('Transport-moving', 1597),
 ('Farming-fishing', 994),
 ('Machine-op-inspct', 2002),
 ('Tech-support', 928),
 ('Protective-serv', 649),
 ('Armed-Forces', 9),
 ('Priv-house-serv', 149)]

但是，正如我们在随机应答中所看到的，翻转比特值产生的"假"应答值将使我们得到难以解释的统计结果。如果我们把扰动后的应答向量逐比特相加，得到的所有计数结果都是错误的：

In [18]:
counts = np.sum([perturb(encode(r)) for r in adult['Occupation']], axis=0)
list(zip(domain, counts))

[('Adm-clerical', 10091),
 ('Exec-managerial', 10380),
 ('Handlers-cleaners', 8738),
 ('Prof-specialty', 10175),
 ('Other-service', 9863),
 ('Sales', 9873),
 ('Craft-repair', 10120),
 ('Transport-moving', 9020),
 ('Farming-fishing', 8718),
 ('Machine-op-inspct', 9163),
 ('Tech-support', 8596),
 ('Protective-serv', 8479),
 ('Armed-Forces', 8160),
 ('Priv-house-serv', 8099)]

In [19]:
def aggregate(responses):
    p = .75
    q = .25
    
    sums = np.sum(responses, axis=0)
    n = len(responses)
    
    return [(v - n*q) / (p-q) for v in sums]  

In [20]:
responses = [perturb(encode(r)) for r in adult['Occupation']]
counts = aggregate(responses)
list(zip(domain, counts))

[('Adm-clerical', 3917.5),
 ('Exec-managerial', 4235.5),
 ('Handlers-cleaners', 1157.5),
 ('Prof-specialty', 4281.5),
 ('Other-service', 3297.5),
 ('Sales', 3715.5),
 ('Craft-repair', 3967.5),
 ('Transport-moving', 1575.5),
 ('Farming-fishing', 827.5),
 ('Machine-op-inspct', 1787.5),
 ('Tech-support', 907.5),
 ('Protective-serv', 555.5),
 ('Armed-Forces', 39.5),
 ('Priv-house-serv', 365.5)]